## Run Example Solvent Extraction Configuration
Install all Install all dependencies using pip install -e .


## Overview
This is a simple test of the solvent extraction code. Same code can be found in front.py

### Import helper modules

In [1]:
""" local imports """
from solventx import solventx as sx
from solventx import utilities as util


### Create and Initialize Solvent Extraction Object
Be sure to update the directory addresses in design.json to match your environment

In [3]:
#
""" build configs """
config_file = ".\\design.json"
confDict = util.read_config(config_file)

config_env_file = ".\\env_design_config.json"
confEnvDict = util.get_env_config_dict(config_env_file)

""" generate cases """
cases = util.generate(confDict,1)

""" select case and create object """
ree_mass = [item for item in cases['0'].values()]
obj = sx.solventx(confDict, confEnvDict, ree_mass)

### Evaluate Simulation and pull example results

In [ ]:
""" evaluate """
obj.evaluate(obj.design_variables) #  

""" Reward """
reward = obj.objective() # normalized reward

""" Examine Specs """
purity = obj.purity
recovery = obj.recovery
recority = obj.recority # recovery * purity

## For Optimization
 You need to install RBFOPT toolbox, pyomo interface for python integration, and your choice of coin-OR project solvers to handle the MINLP problem

### Run Optimization
The result is returned in the form of a dictionary (see optimize() function above)
Remember to update the addresses to match your local directory system

In [ ]:
""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
""" 
Section B: Call the Standard Optimizer
"""    
""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
t0 = time.time()

config_file = ".\\design.json"
confDict = util.read_config(config_file)

config_env_file = ".\\env_design_config.json"
confEnvDict = util.get_env_config_dict(config_env_file)

""" generate cases """
cases = util.generate(confDict,3)

t1 = time.time()
#for now, pick only the first cases - later cases will use parallel submission
ree_mass = [item for item in cases['0'].values()]

# instantiate solvent extraction object
obj = sx.solventx(confDict, confEnvDict, ree_mass) 
obj.cases = cases['0']
iters = 100 # Number of iteration, at least 100 
    
resjsn = optimize(obj, iters)
#

with open('solventx/data/json/results.json', 'w') as json_file:
    json.dump(resjsn, json_file)